In [19]:
######tensorflow基础
#huanshuo0801@gmail.com

In [28]:
import tensorflow as tf
import os
import numpy as np

In [5]:
#操作之间的依赖关系
a = tf.Variable(1.0,name="a") #定义变量a
b = tf.add(a,1,name="b") #定义操作a+1
c = tf.add(b,1,name="c") #定义操作b+1
d = tf.add(b,10,name="d") #定义操作b+10

In [7]:
#会话中执行计算 1+1
#构建图，定义两个常量和加法操作
v1 = tf.constant(1,name="value1")
v2 = tf.constant(1,name="value2")
add_op = tf.add(v1,v2,name="add_op_name") #只是定义加法操作，但是并没有执行

#在会话中执行操作
with tf.Session() as sess:
    #执行运算，将Tensorflow中的运算结果赋给python中的变量
    result = sess.run(add_op)  #加法结果赋值给result
    print("1 + 1 = %.0f" % result)

1 + 1 = 2


In [16]:
#检查运算
tf.reset_default_graph() # 重置图
v1 = tf.constant(1,name="value1")
v2 = tf.constant(1,name="value2")
add_op = tf.add(v1,v2,name="add_op_name") #只是定义加法操作，但是并没有执行

graph = tf.get_default_graph() #获取默认图
operations = graph.get_operations() #获取图中所有操作
#也可以这样写 tf.get_default_graph().get_operations()

print("number of operations : %d" % len(operations))
print("operations: ")
for op in operations:
    print(op) # 打印操作，可以看到一共有三个操作

number of operations : 3
operations: 
name: "value1"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
      }
      int_val: 1
    }
  }
}

name: "value2"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
      }
      int_val: 1
    }
  }
}

name: "add_op_name"
op: "Add"
input: "value1"
input: "value2"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [24]:
#会话中计算的依赖
tf.reset_default_graph() # 重置图
x = tf.Variable(0.0,name="x") #定义变量x
x_plus_1 = tf.assign_add(x, 1, name="x_plus") #定义操作 将x+1后的值赋给x

with tf.control_dependencies([x_plus_1]): #依赖x_plus_1的操作
    y = tf.identity(x,name='y') #定义操作y 返回一个和x拥有同样的值，相同形状的tensor
    
init = tf.global_variables_initializer() #初始化所有变量

with tf.Session() as sess:
    sess.run(init)
    for i in range(5):
        print(y.eval())

        
#保存图结构，后面可以用tensorboard查看
summary_writer = tf.summary.FileWriter('.calc_graph')
graph = tf.get_default_graph()
summary_writer.add_graph(graph)
summary_writer.flush()

1.0
2.0
3.0
4.0
5.0


In [25]:
#变量的初始化
Weight1 = tf.Variable(0.001) #定义变量Weight1
Weight2 = tf.Variable(Weight1.initialized_value() * 2) #Weight2的初始值是Weight1的2倍

init = tf.global_variables_initializer() #变量必须初始化！
with tf.Session() as sess:
    sess.run(init)
    print("Weight1 is:")
    print(sess.run(Weight1))
    print("Weight2 is:")
    print(sess.run(Weight2))

Weight1 is:
0.001
Weight2 is:
0.002


In [26]:
#变量的变形
v = tf.Variable([1,2,3,4,5,6,7,8,9]) #定一个长度为9的向量

reshaped_v = tf.reshape(v,[3,3]) #变成3*3

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print("v's value is:")
    print(sess.run(v))
    print("reshaped value is:")
    print(sess.run(reshaped_v))

v's value is:
[1 2 3 4 5 6 7 8 9]
reshaped value is:
[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [27]:
#数据类型和维度
#https://www.cnblogs.com/tengge/p/6360946.html
value_shape0 = tf.Variable(6666)
value_shape1 = tf.Variable([1.1,2.2,3.3])
value_shape2 = tf.Variable([1,2],[3,4],[5,6])
value_shape3 = tf.Variable([[1],[2],[3]],[[4],[5],[6]]) # 第一维度是2 第二维度是3 第三维度是1

print(value_shape0.get_shape())
print(value_shape1.get_shape())
print(value_shape2.get_shape())
print(value_shape3.get_shape())

()
(3,)
(2,)
(3, 1)


In [43]:
############训练Y=weight*X + bias 
def get_data(number):
    '''获取训练数据和测试数据'''
    list_x = []
    list_label = []
    for i in range(number):
        x = np.random.randn(1) #随机产生x
        label = 2 * x + np.random.randn(1) * 0.01 + 10
        #将x和label都放入list ，当作样本的特征和label
        list_x.append(x) 
        list_label.append(label)
    return list_x,list_label


def inference(x):
    '''定义网络计算结构 输入x 输出y=weight*x+bias'''
    weight = tf.Variable("weight",[1])
    bias = tf.Variable("bias",[1])
    y = x * weight + bias
    return y

train_x = tf.placeholder(tf.float32)
train_label = tf.placeholder(tf.float32)
test_x = tf.placeholder(tf.float32)
test_label = tf.placeholder(tf.float32)

with tf.variable_scope("inference"):
    train_y  = inference(train_x)
    tf.variable_scope().reuse_variablies()
    test_y = inference(test_x)
    
'''
定义损失函数来计算损失值，这里利用差的平方作为损失函数
损失函数是用来计算训练的结果和很是结果的差多少的方法
在正常的训练过程中，损失值会越来越少，模型会越来越模拟输入训练数据的分布
'''
train_loss = tf.square(train_y - train_label)
test_loss = tf.square(test_y - test_label)

opt = tf.train.GradientDescentOptimizer(0.002) #采用梯度下降优化函数，根据损失函数的值，确定参数更新方向
train_op = opt.minimize(train_loss)

init = tf.global_variables_initializer() #参数初始化

train_data_x,train_data_label = get_data(1000) #读取训练输出
test_data_x,test_data_label = get_data(1)  #读取测试数据

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(1000):
        #运行一次训练操作
        sess.run(train_op,feed_dict={train_x:train_data_x[i],
                                     train_label:train_data_label[i]})
        if i % 10 == 0:
            #每运行10次，测试一下数据
            test_loss_value = sess.run(test_loss,
                    feed_dict={test_x:test_data_x[0],
                               test_label:test_data_label[0]})
            print("step %d eval loss is %.3f" % (i,test_loss_value))

ValueError: Incompatible type conversion requested to type 'float32' for variable of type 'string_ref'

In [41]:
import numpy as np
import tensorflow as tf

# 获取训练数据和测试数据
def get_data(number):
    list_x = []
    list_label = []
    for i in range(number):
        x = np.random.randn(1)
        # 这里构建数据的分布满足 y = 2 * x + 10
        label = 2 * x + np.random.randn(1) * 0.01  + 10
        list_x.append(x)
        list_label.append(label)
    return list_x, list_label

def inference(x):
    weight = tf.get_variable("weight",[1])
    bias = tf.get_variable("bias",[1])
    y = x * weight + bias
    return y

train_x = tf.placeholder(tf.float32)
train_label = tf.placeholder(tf.float32)
test_x = tf.placeholder(tf.float32)
test_label = tf.placeholder(tf.float32)

with tf.variable_scope("inference"):
    train_y = inference(train_x)
    tf.get_variable_scope().reuse_variables()
    test_y = inference(test_x)

train_loss = tf.square(train_y - train_label)
test_loss = tf.square(test_y - test_label)
opt = tf.train.GradientDescentOptimizer(0.002)
train_op = opt.minimize(train_loss)

init = tf.global_variables_initializer()

train_data_x, train_data_label = get_data(1000) #读取训练数据的函数
test_data_x, test_data_label = get_data(1)

with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        sess.run(train_op, feed_dict={train_x: train_data_x[i],
                                      train_label:train_data_label[i]})
        if i % 10 == 0:
            test_loss_value = sess.run(test_loss, 
                            feed_dict={test_x:test_data_x[0],
                                       test_label: test_data_label[0]})
            print("step %d eval loss is %.3f" %(i,test_loss_value))

ValueError: Variable inference/weight already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-38-1d78d643e328>", line 17, in inference
    weight = tf.get_variable("weight",[1])
  File "<ipython-input-38-1d78d643e328>", line 28, in <module>
    train_y  = inference(train_x)
  File "/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
